In [1]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore")

data = pd.read_csv("/content/drive/My Drive/Data/jobkorea_all.csv")

In [ ]:
import os

os.chdir("/content/drive/My Drive/preprocess")

In [ ]:
from preprocess import _preprocess_company, _preprocess_qna

# 회사명, 답변 전처리
data = _preprocess_company(data)
data = _preprocess_qna(data)

In [ ]:
# 회사명을 단어 사전에 추가
# 단어 사전에 추가하는 방법이 가장 간단해서 Komoran 사용
company_dict = pd.DataFrame({"회사명": data["회사명"].unique()})
company_dict["품사"] = "N"
company_dict.to_csv("company_dict.txt", sep="\t", index=False, header=False)

In [ ]:
from konlpy.tag import Komoran

komoran = Komoran(userdic="company_dict.txt")

In [ ]:
data["답변"] = data["답변"].apply(lambda x: komoran.morphs(x))
data["답변"] = data["답변"].apply(lambda x: " ".join(x))

In [3]:
from gensim.models import FastText

# CBOW 방식: 앞과 뒤에 위치한 단어들로 가운데 위치한 단어를 예측하는 과정에서 임베딩
# -> 앞에 위치한 단어들만으로 뒤에 올 단어를 예측하는 방식으로 변경해서 임베딩하면 자기소개서 막혔을 때 다음 단어를 추천하는데 활용할 수 있을 듯

model = FastText(data["답변"].to_list(), size=1000, sg=0, alpha=0.05, window=5, min_count=5, min_alpha=1e-4, batch_words=512)
model.save("/content/drive/My Drive/TMI_FastText_CBOW.vec")
# model = FastText.load("/content/drive/My Drive/TMI_FastText_CBOW.vec")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [20]:
model.wv.similar_by_word('ai')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('kai', 0.7183406352996826),
 ('정보통신기술', 0.6594484448432922),
 ('커넥티드', 0.6578778028488159),
 ('aicc', 0.6542792916297913),
 ('iot', 0.6536335349082947),
 ('ioe', 0.6456980109214783),
 ('블록체인', 0.6434512734413147),
 ('커넥티드카', 0.6310164928436279),
 ('aim', 0.625775933265686),
 ('핀테크', 0.6254277229309082)]

In [21]:
# FastText의 임베딩 특성 상 오탈자나 훈련용 말뭉치에 없는 단어에 대해서도 활용이 가능
model.wv.similar_by_word("인공지눙")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('인공지능', 0.9922890663146973),
 ('인공', 0.7571531534194946),
 ('ai', 0.6029238700866699),
 ('iot', 0.5973562598228455),
 ('ioe', 0.5778467655181885),
 ('iiot', 0.5604372620582581),
 ('핀테크', 0.5127812027931213),
 ('알파고', 0.5055826902389526),
 ('인공신경망', 0.5038849115371704),
 ('컴퓨팅', 0.4940435290336609)]